In [1]:
import pandas as pd
import os

cwd = os.getcwd() # change for '__file__' if using .py 

stats = pd.read_csv((cwd + '\player_mvp_stats.csv'))  
del stats['Unnamed: 0']
pd.isnull(stats).sum()                                              # check for missing values
stats[pd.isnull(stats['3P%'])][['Player','3PA']]                    # check the columns Player and 3PA for anyvalue where the 3P% is null (likely due to 3PA == 0)
stats = stats.fillna(0)

ModuleNotFoundError: No module named 'pandas'

In [20]:
n_Nan = sum(pd.isnull(stats).sum())
if n_Nan > 0:
    print('\x1b[7;31;40m' + 'there are some "Nan" values here' + '\x1b[0m')                                          # check for missing values
    print(('n = ' + str(n_Nan)))
else:
    print('\x1b[3;30;42m' + 'no NaNs found' + '\x1b[0m')     # check more colours in https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal


no NaNs found


In [28]:
stats.columns
# trying to predict share so remove the variablkes that directly relate to it ('Pts Won', 'Pts Max', 'Share')

predictors = ['Age_x', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Age_y', 'W', 'L', 'W/L%', 'GB',
       'PS/G', 'PA/G', 'SRS']

train = stats[stats['Year'] < 2021]
test  = stats[stats['Year'] == 2021]

In [66]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha = .5)   # this how much does the algorithm prevent overfitting
reg.fit(train[predictors],train['Share'])
predictions = reg.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=['predictions'], index = test.index)

results = pd.concat([test[['Player','Share']],predictions], axis=1)  # add the columns Player, Share, Preditions

results = results.sort_values('Share', ascending = False)                                       # create rank columns for actual observations and predicted dats
results['Rk'] = list(range(1,results.shape[0]+1))

results = results.sort_values('predictions', ascending = False)
results['Rk_predicted'] = list(range(1,results.shape[0]+1))

results.head(10)

,Player,Share,predictions,Rk,Rk_predicted
3736,LeBron James,0.001,0.240651,15,1
9907,Giannis Antetokounmpo,0.345,0.234324,4,2
3651,Stephen Curry,0.449,0.222280,3,3
4174,James Harden,0.001,0.206627,13,4
3876,Russell Westbrook,0.005,0.199391,11,5
8624,Joel Embiid,0.580,0.198365,2,6
7464,Damian Lillard,0.038,0.191812,7,7
641,Nikola Jokić,0.961,0.189447,1,8
1389,Chris Paul,0.138,0.187032,5,9
5328,Kawhi Leonard,0.001,0.186364,14,10


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(results[]['Share'],results['predictions'])       # this is not very useful because we want the rank 